In [3]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from functions.functions_to_import import display_widgets

df = pd.read_csv("cleaned_zomato.csv")
df["cuisines"] = df["cuisines"].apply(eval)

ModuleNotFoundError: No module named 'functions_to_import'

In [ ]:
display_widgets(df)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12066 entries, 0 to 12065
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   url        12066 non-null  object 
 1   address    12066 non-null  object 
 2   name       12066 non-null  object 
 3   rate       9061 non-null   float64
 4   votes      12066 non-null  int64  
 5   phone      12066 non-null  object 
 6   location   12066 non-null  object 
 7   rest_type  12010 non-null  object 
 8   cuisines   12066 non-null  object 
 9   cost       12019 non-null  float64
 10  latitude   12066 non-null  float64
 11  longitude  12066 non-null  float64
dtypes: float64(4), int64(1), object(7)
memory usage: 1.1+ MB


In [82]:
def run_application(preferences,df:pd.DataFrame):

    filtered_data = filter_data(preferences,df)
    user_location=generate_user_location(filtered_data)
    data_with_distance=calculate_distance_and_walk_time(filtered_data, user_location)
    normalized_data=normalize_data(data_with_distance,['rate','cost','distance','votes'])
    final_data=calculate_euclidean_distance(normalized_data)
    top_10_restaurants = final_data.sort_values(by="euclidean_dist",ascending=True)[:10]
    # map=generate_map(top_10_restaurants, user_location)
    # display(map)
    
    

In [14]:
# Types of cuisines
def cuisines_list(df):
    
    global dict_of_cuisines
    dict_of_cuisines = {}
    for x in df.index:
        for y in df.loc[x,'cuisines']:
            if y not in dict_of_cuisines.keys():
                dict_of_cuisines[y] = 1
            else: 
                dict_of_cuisines[y] += 1
    dict_of_cuisines = dict(sorted(dict_of_cuisines.items(), key=lambda item: item[1], reverse = True))
    return dict_of_cuisines

In [33]:
def display_widgets():

    price_selection = widgets.SelectionRangeSlider(options=['$', '$$', '$$$','$$$$'],index=(0, 3), 
                                                    description='Price range: ',disabled=False)
    cuisine_selection = widgets.SelectMultiple(options=cuisines_list(df).keys(),
                                                description='Cuisines',disabled=False)
    rate_selection = widgets.FloatSlider(value=4,min=0,max=5,step=0.1, description='Minimum rate: ')

    button = widgets.Button(description="OK")
    
    global preferences
    preferences={}

    def collect_preferences():

        preferences["price"] = price_selection.value
        preferences["cuisine"] = cuisine_selection.value
        preferences["rate"] = rate_selection.value
        return preferences

    def display_preferences(preferences):
        print(f'Your preferences:\n -- cuisine: {preferences["cuisine"]} \n -- minimum rating: {preferences["rate"]} \n -- price: {preferences["price"]}')


    def on_button_clicked(b):

        collect_preferences()
        price_selection.close()
        cuisine_selection.close()
        rate_selection.close()
        button.close()
        display_preferences(preferences)
        run_application()

    display(cuisine_selection,rate_selection,price_selection, button)
    button.on_click(on_button_clicked)

In [31]:
display_widgets()

SelectMultiple(description='Cuisines', options=('North Indian', 'Chinese', 'South Indian', 'Fast Food', 'Birya…

FloatSlider(value=4.0, description='Minimum rate: ', max=5.0)

SelectionRangeSlider(description='Price range: ', index=(0, 3), options=('$', '$$', '$$$', '$$$$'), value=('$'…

Button(description='OK', style=ButtonStyle())

Your preferences:
 -- cuisine: ('Chinese', 'South Indian', 'Fast Food') 
 -- rating: 3.9 
 -- price: ('$$', '$$$$')


In [38]:
# Price filter:
import numpy as np

def price_filter(preferences:dict,df:pd.DataFrame):

    # Ranges were defined based on the following analysis of the input data frame:
    q25 = df["cost"].quantile(0.25)
    q50 = df["cost"].quantile(0.5)
    q75 = df["cost"].quantile(0.75)
    max = df["cost"].max()

    p = preferences["price"]
    price_ranges={'$':(0,q25), '$$': (q25,q50), '$$$':(q50,q75), '$$$$':(q75,max)}
    price_min = float(price_ranges[p[0]][0])
    price_max = float(price_ranges[p[1]][1])
    df.query("@price_min < cost <= @price_max",inplace=True)

    return df

In [26]:
# Cuisine filter
def cuisines_filter(preferences:dict,df:pd.DataFrame): 
    
    p = preferences["cuisine"]
    
    df.loc[(df.apply(lambda x: any(m in str(v)
                               for v in x.values 
                               for m in p), axis=1))]
    return df

In [36]:
# Range filter
def range_filter(preferences:dict,df:pd.DataFrame):
    
    p = preferences["rate"]
    df.query(f'rate >= {p}', inplace=True)
    return df


In [61]:
df2 = df.copy()

In [62]:

def filter_data(preferences,df):
    cuisines_filter(preferences, df)
    price_filter(preferences, df)
    range_filter(preferences, df)
    
    return df

In [63]:
filtered_data = filter_data(preferences,df2)
filtered_data

,url,address,name,rate,votes,phone,location,rest_type,cuisines,cost,latitude,longitude
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,"080 42297555, +91 9743772233",Banashankari,Casual Dining,"[North Indian, Mughlai, Chinese]",800.0,12.968542,77.599406
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Banashankari,Casual Dining,"[Chinese, North Indian, Thai]",800.0,12.967906,77.592121
7,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,4.6,2556,"080 48653961, 080 48655715",Banashankari,"Casual Dining, Cafe","[Pizza, Cafe, Italian]",600.0,12.969211,77.592705
8,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,4.0,324,"+91 8884135549, +91 9449449316",Banashankari,Cafe,"[Cafe, Italian, Continental]",700.0,12.964422,77.594771
9,https://www.zomato.com/bangalore/smacznego-ban...,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,4.2,504,"+91 9945230807, +91 9743804471",Banashankari,Cafe,"[Cafe, Mexican, Italian, Momos, Beverages]",550.0,12.964658,77.595155
...,...,...,...,...,...,...,...,...,...,...,...,...
12002,https://www.zomato.com/bangalore/kfc-2-whitefi...,"Unit No.UG-35 B, 2nd Floor, Phoenix Market Cit...",KFC,3.9,167,+91 8033994444,Whitefield,"Food Court, Quick Bites","[Burger, Fast Food]",450.0,12.973074,77.748520
12010,https://www.zomato.com/bangalore/nu-tree-1-whi...,"12th Floor, Gamma Building, Sigma Soft Tech Pa...",nu.tree,4.2,52,+91 8310393035,Whitefield,Quick Bites,"[North Indian, Healthy Food, Beverages]",400.0,12.970316,77.749328
12056,https://www.zomato.com/bangalore/chianti-white...,"1, Ground Floor, Ascendas Park Square Mall, Wh...",Chianti,4.6,848,"+91 8061224716, +91 8061224717","ITPL Main Road, Whitefield",Casual Dining,[Italian],1500.0,12.978196,77.732199
12057,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,4.3,236,080 49652769,"ITPL Main Road, Whitefield",Bar,[Finger Food],2500.0,12.976141,77.724823


In [64]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2493 entries, 0 to 12062
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   url        2493 non-null   object 
 1   address    2493 non-null   object 
 2   name       2493 non-null   object 
 3   rate       2493 non-null   float64
 4   votes      2493 non-null   int64  
 5   phone      2493 non-null   object 
 6   location   2493 non-null   object 
 7   rest_type  2489 non-null   object 
 8   cuisines   2493 non-null   object 
 9   cost       2493 non-null   float64
 10  latitude   2493 non-null   float64
 11  longitude  2493 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 253.2+ KB


In [65]:
import random
# Wywolosowanie user_location
def generate_user_location(df:pd.DataFrame):
    
        lat = random.uniform(df["latitude"].min(), df["latitude"].max())
        long = random.uniform(df["longitude"].min(), df["longitude"].max())
        global user_location
        user_location = lat, long
        return user_location


In [66]:
user_location=generate_user_location(filtered_data)
user_location

(13.044569236116962, 77.68661011809586)

In [67]:
# Obliczanie odległości
from  geopy import distance
def calculate_distance_and_walk_time(df, user_location):
    df_latitude_col=df["latitude"]
    df_longitude_col=df["longitude"]
    rest_location = list(zip(df_latitude_col, df_longitude_col))
    df['rest_location'] = rest_location
    df['distance'] = df['rest_location'].apply(lambda x: distance.distance(x, user_location).kilometers)
    df['walk_time_estimation'] = round(df['distance'].apply(lambda x: x * 12),0)
    return df

In [68]:
data_with_distance=calculate_distance_and_walk_time(filtered_data, user_location)
data_with_distance

,url,address,name,rate,votes,phone,location,rest_type,cuisines,cost,latitude,longitude,rest_location,distance,walk_time_estimation
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,"080 42297555, +91 9743772233",Banashankari,Casual Dining,"[North Indian, Mughlai, Chinese]",800.0,12.968542,77.599406,"(12.96854225, 77.59940595)",12.658450,152.0
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Banashankari,Casual Dining,"[Chinese, North Indian, Thai]",800.0,12.967906,77.592121,"(12.96790625, 77.59212095000001)",13.304215,160.0
7,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,4.6,2556,"080 48653961, 080 48655715",Banashankari,"Casual Dining, Cafe","[Pizza, Cafe, Italian]",600.0,12.969211,77.592705,"(12.96921125, 77.59270495000001)",13.163538,158.0
8,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,4.0,324,"+91 8884135549, +91 9449449316",Banashankari,Cafe,"[Cafe, Italian, Continental]",700.0,12.964422,77.594771,"(12.964422249999998, 77.59477095000001)",13.337135,160.0
9,https://www.zomato.com/bangalore/smacznego-ban...,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,4.2,504,"+91 9945230807, +91 9743804471",Banashankari,Cafe,"[Cafe, Mexican, Italian, Momos, Beverages]",550.0,12.964658,77.595155,"(12.96465825, 77.59515495000001)",13.288659,159.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12002,https://www.zomato.com/bangalore/kfc-2-whitefi...,"Unit No.UG-35 B, 2nd Floor, Phoenix Market Cit...",KFC,3.9,167,+91 8033994444,Whitefield,"Food Court, Quick Bites","[Burger, Fast Food]",450.0,12.973074,77.748520,"(12.9730745, 77.7485198)",10.376182,125.0
12010,https://www.zomato.com/bangalore/nu-tree-1-whi...,"12th Floor, Gamma Building, Sigma Soft Tech Pa...",nu.tree,4.2,52,+91 8310393035,Whitefield,Quick Bites,"[North Indian, Healthy Food, Beverages]",400.0,12.970316,77.749328,"(12.9703165, 77.7493278)",10.666323,128.0
12056,https://www.zomato.com/bangalore/chianti-white...,"1, Ground Floor, Ascendas Park Square Mall, Wh...",Chianti,4.6,848,"+91 8061224716, +91 8061224717","ITPL Main Road, Whitefield",Casual Dining,[Italian],1500.0,12.978196,77.732199,"(12.9781964, 77.7321988)",8.852975,106.0
12057,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,4.3,236,080 49652769,"ITPL Main Road, Whitefield",Bar,[Finger Food],2500.0,12.976141,77.724823,"(12.9761414, 77.7248228)",8.630862,104.0


In [54]:
# Data normalization

In [69]:
# Data normalization
def normalize_data (df, column_names:list):

    for column_name in column_names:
        new_name = "".join([column_name, "norm"])
        df[new_name] = 0
        x_min = df[column_name].min()
        x_max = df[column_name].max()
        for x in df[column_name].index: 
            df.at[x, new_name] = (df[column_name][x] - x_min)/(x_max-x_min)
    
    return df

In [56]:
distance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2493 entries, 0 to 12062
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   url                   2493 non-null   object 
 1   address               2493 non-null   object 
 2   name                  2493 non-null   object 
 3   rate                  2493 non-null   float64
 4   votes                 2493 non-null   int64  
 5   phone                 2493 non-null   object 
 6   location              2493 non-null   object 
 7   rest_type             2489 non-null   object 
 8   cuisines              2493 non-null   object 
 9   cost                  2493 non-null   float64
 10  latitude              2493 non-null   float64
 11  longitude             2493 non-null   float64
 12  rest_location         2493 non-null   object 
 13  distance              2493 non-null   float64
 14  walk_time_estimation  2493 non-null   float64
dtypes: float64(6), int64

In [70]:
normalized_data=normalize_data(data_with_distance,['rate','cost','distance','votes'])


In [72]:
normalized_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2493 entries, 0 to 12062
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   url                   2493 non-null   object 
 1   address               2493 non-null   object 
 2   name                  2493 non-null   object 
 3   rate                  2493 non-null   float64
 4   votes                 2493 non-null   int64  
 5   phone                 2493 non-null   object 
 6   location              2493 non-null   object 
 7   rest_type             2489 non-null   object 
 8   cuisines              2493 non-null   object 
 9   cost                  2493 non-null   float64
 10  latitude              2493 non-null   float64
 11  longitude             2493 non-null   float64
 12  rest_location         2493 non-null   object 
 13  distance              2493 non-null   float64
 14  walk_time_estimation  2493 non-null   float64
 15  ratenorm            

In [76]:
from scipy.spatial import distance as scipy_distance

# Obliczanie odległości euklidesowych - wybór optymalnych restauracji
def calculate_euclidean_distance(df):
    df_rate_col = df["ratenorm"]
    df_votes_col = df["votesnorm"]
    df_cost_col = df["costnorm"]
    df_dist_col= df["distancenorm"]
    df['rest_score'] = list(zip(df_rate_col, df_votes_col, df_cost_col, df_dist_col))
    df['euclidean_dist'] = df['rest_score'].apply(lambda rest_score_col: scipy_distance.euclidean(rest_score_col, [1, 1, 0, 0],w=[1, 0.5, 1, 3] ))
    return df

In [77]:
final_data=calculate_euclidean_distance(normalized_data)
final_data

,url,address,name,rate,votes,phone,location,rest_type,cuisines,cost,...,longitude,rest_location,distance,walk_time_estimation,ratenorm,costnorm,distancenorm,votesnorm,rest_score,euclidean_dist
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,4.1,775,"080 42297555, +91 9743772233",Banashankari,Casual Dining,"[North Indian, Mughlai, Chinese]",800.0,...,77.599406,"(12.96854225, 77.59940595)",12.658450,152.0,0.2,0.119048,0.453546,0.047415,"(0.19999999999999965, 0.04741511165494035, 0.1...",1.313390
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,4.1,787,080 41714161,Banashankari,Casual Dining,"[Chinese, North Indian, Thai]",800.0,...,77.592121,"(12.96790625, 77.59212095000001)",13.304215,160.0,0.2,0.119048,0.483220,0.048149,"(0.19999999999999965, 0.04814928112572652, 0.1...",1.344503
7,https://www.zomato.com/bangalore/onesta-banash...,"2469, 3rd Floor, 24th Cross, Opposite BDA Comp...",Onesta,4.6,2556,"080 48653961, 080 48655715",Banashankari,"Casual Dining, Cafe","[Pizza, Cafe, Italian]",600.0,...,77.592705,"(12.96921125, 77.59270495000001)",13.163538,158.0,0.7,0.071429,0.476756,0.156378,"(0.6999999999999994, 0.15637809727745489, 0.07...",1.064350
8,https://www.zomato.com/bangalore/penthouse-caf...,"1, 30th Main Road, 3rd Stage, Banashankari, Ba...",Penthouse Cafe,4.0,324,"+91 8884135549, +91 9449449316",Banashankari,Cafe,"[Cafe, Italian, Continental]",700.0,...,77.594771,"(12.964422249999998, 77.59477095000001)",13.337135,160.0,0.1,0.095238,0.484733,0.019823,"(0.10000000000000005, 0.019822575711226673, 0....",1.415748
9,https://www.zomato.com/bangalore/smacznego-ban...,"2470, 21 Main Road, 25th Cross, Banashankari, ...",Smacznego,4.2,504,"+91 9945230807, +91 9743804471",Banashankari,Cafe,"[Cafe, Mexican, Italian, Momos, Beverages]",550.0,...,77.595155,"(12.96465825, 77.59515495000001)",13.288659,159.0,0.3,0.059524,0.482506,0.030835,"(0.30000000000000016, 0.030835117773019272, 0....",1.289038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12002,https://www.zomato.com/bangalore/kfc-2-whitefi...,"Unit No.UG-35 B, 2nd Floor, Phoenix Market Cit...",KFC,3.9,167,+91 8033994444,Whitefield,"Food Court, Quick Bites","[Burger, Fast Food]",450.0,...,77.748520,"(12.9730745, 77.7485198)",10.376182,125.0,0.0,0.035714,0.348672,0.010217,"(0.0, 0.010217191801774243, 0.0357142857142857...",1.362287
12010,https://www.zomato.com/bangalore/nu-tree-1-whi...,"12th Floor, Gamma Building, Sigma Soft Tech Pa...",nu.tree,4.2,52,+91 8310393035,Whitefield,Quick Bites,"[North Indian, Healthy Food, Beverages]",400.0,...,77.749328,"(12.9703165, 77.7493278)",10.666323,128.0,0.3,0.023810,0.362004,0.003181,"(0.30000000000000016, 0.0031814010400734167, 0...",1.174960
12056,https://www.zomato.com/bangalore/chianti-white...,"1, Ground Floor, Ascendas Park Square Mall, Wh...",Chianti,4.6,848,"+91 8061224716, +91 8061224717","ITPL Main Road, Whitefield",Casual Dining,[Italian],1500.0,...,77.732199,"(12.9781964, 77.7321988)",8.852975,106.0,0.7,0.285714,0.278677,0.051881,"(0.6999999999999994, 0.05188130926888957, 0.28...",0.924165
12057,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,4.3,236,080 49652769,"ITPL Main Road, Whitefield",Bar,[Finger Food],2500.0,...,77.724823,"(12.9761414, 77.7248228)",8.630862,104.0,0.4,0.523810,0.268471,0.014439,"(0.39999999999999974, 0.014438666258794738, 0....",1.155972


In [80]:
top_10_restaurants = final_data.sort_values(by="euclidean_dist",ascending=True)[:10]

In [81]:
top_10_restaurants

,url,address,name,rate,votes,phone,location,rest_type,cuisines,cost,...,longitude,rest_location,distance,walk_time_estimation,ratenorm,costnorm,distancenorm,votesnorm,rest_score,euclidean_dist
8223,https://www.zomato.com/bangalore/toit-indirana...,"298, Namma Metro Pillar 62, 100 Feet Road, Ind...",Toit,4.7,14956,+91 9019713388,Indiranagar,Microbrewery,"[Italian, American, Pizza]",1500.0,...,77.644742,"(12.9689773, 77.64474220000001)",9.516562,114.0,0.8,0.285714,0.309170,0.915020,"(0.7999999999999999, 0.9150198837565005, 0.285...",0.641874
10490,https://www.zomato.com/bangalore/abs-absolute-...,"90/4, 3rd Floor, Outer Ring Road, Munnekollaly...",AB's - Absolute Barbecues,4.8,12121,080 49652574,Marathahalli,Casual Dining,"[European, Mediterranean, North Indian, BBQ]",1600.0,...,77.702402,"(12.9602402, 77.7024023)",9.485361,114.0,0.9,0.309524,0.307737,0.741572,"(0.8999999999999995, 0.7415723462832671, 0.309...",0.650617
3569,https://www.zomato.com/bangalore/communiti-res...,"67 & 68, Brigade Solitaire, Opposite to Advait...",Communiti,4.7,3894,"080 40999755\r, +91 9342158585",Residency Road,"Microbrewery, Casual Dining","[Continental, BBQ, Salad]",1500.0,...,77.623819,"(13.0515796, 77.6238191)",6.854572,82.0,0.8,0.285714,0.186847,0.238238,"(0.7999999999999999, 0.2382379932701132, 0.285...",0.718685
8690,https://www.zomato.com/bangalore/belgian-waffl...,"5M/406, Ground Floor, 2nd Block extension, Hen...",Belgian Waffle Factory,4.8,265,+91 9481511911,Kalyan Nagar,Dessert Parlor,[Desserts],400.0,...,77.636171,"(13.0239136, 77.6361708)",5.929194,71.0,0.9,0.023810,0.144324,0.016213,"(0.8999999999999995, 0.01621290914652799, 0.02...",0.746307
2030,https://www.zomato.com/bangalore/the-black-pea...,"20/7, Swamy Legato, Outer Ring Road, Kadubeesa...",The Black Pearl,4.8,7023,080 49653069,Marathahalli,"Casual Dining, Bar","[North Indian, European, Mediterranean, BBQ]",1500.0,...,77.696821,"(12.9546072, 77.69682130000001)",10.014003,120.0,0.9,0.285714,0.332029,0.429673,"(0.8999999999999995, 0.42967268277760784, 0.28...",0.764852
7636,https://www.zomato.com/bangalore/barbeque-nati...,"4005, HAL 2nd Stage, 100 Feet Road, Indiranaga...",Barbeque Nation,4.7,7152,+91 8064806060,Indiranagar,Casual Dining,"[North Indian, European, Mediterranean, BBQ, K...",1600.0,...,77.636480,"(12.9724823, 77.63648020000001)",9.652686,116.0,0.8,0.309524,0.315426,0.437565,"(0.7999999999999999, 0.4375650045885592, 0.309...",0.769709
7635,https://www.zomato.com/bangalore/big-pitcher-a...,"LR Arcade,4121, Old Airport Road, Bangalore",Big Pitcher,4.7,9021,080 49652494,Old Airport Road,"Pub, Microbrewery","[American, Continental, North Indian, Mediterr...",1800.0,...,77.662460,"(12.9560298, 77.6624596)",10.139496,122.0,0.8,0.357143,0.337795,0.551912,"(0.7999999999999999, 0.5519118996635056, 0.357...",0.781191
4033,https://www.zomato.com/bangalore/brew-and-barb...,"36/4, Fourth Floor, Soul Space Arena, Outer Ri...",Brew and Barbeque - A Microbrewery Pub,4.7,5746,080 45128649,Marathahalli,"Microbrewery, Pub","[Continental, North Indian, BBQ, Steak]",1400.0,...,77.699219,"(12.9583802, 77.69921930000001)",9.632754,116.0,0.8,0.261905,0.314510,0.351545,"(0.7999999999999999, 0.35154481492811257, 0.26...",0.784596
8190,https://www.zomato.com/bangalore/milano-ice-cr...,"460, 2nd Cross, Krishna Temple Road, Indiranag...",Milano Ice Cream,4.9,2090,"080 41256483, +91 8281464593",Indiranagar,Dessert Parlor,"[Ice Cream, Desserts]",400.0,...,77.640257,"(12.9779143, 77.64025720000001)",8.925321,107.0,1.0,0.023810,0.282002,0.127868,"(1.0, 0.12786784949525848, 0.02380952380952380...",0.787051
8609,https://www.zomato.com/bangalore/abs-absolute-...,"2nd Floor, Roseline Landmark, 5th B Cross, HRB...",AB's - Absolute Barbecues,4.6,1502,080 30474040,Kalyan Nagar,Casual Dining,"[European, Mediterranean, North Indian, BBQ]",1400.0,...,77.638660,"(13.0214796, 77.6386598)",5.794606,70.0,0.7,0.261905,0.138139,0.091894,"(0.6999999999999994, 0.091893545426736, 0.2619...",0.792572
